In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker
import numpy as np
import random
import time
import os
import pandas as pd
import math
import bcolz
import pickle
import re

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
print("Pytorch: {}".format(torch.__version__))

Pytorch: 0.2.0_4


# Data loading

In [3]:
all_data_org = pd.read_csv('data/en_train_org.csv')

In [4]:
all_data_org[pd.isnull(all_data_org['before'])][:3]

,sentence_id,token_id,class,before,after
616107,49226,17,LETTERS,NaN,n a
684691,54634,1,PLAIN,NaN,NaN
965529,76612,7,PLAIN,NaN,NaN


In [5]:
all_data = all_data_org.dropna()
print("Data rows: {},  (dropped none rows: {})".format(len(all_data), len(all_data_org)-len(all_data)))
all_data = all_data[all_data['class'] != 'VERBATIM']
print("Data rows: {},  (dropped rows: {})".format(len(all_data), len(all_data_org)-len(all_data)))
all_data = all_data.reset_index(drop=True)

Data rows: 9918390,  (dropped none rows: 51)
Data rows: 9840282,  (dropped rows: 78159)


Note we dropped VERBATIM class. Thats because it had so many weird characters.

In [6]:
all_data_sentence_index = all_data.set_index('sentence_id')

### More balanced sample

In [7]:
all_data.groupby("class")["class"].count()

class
ADDRESS           522
CARDINAL       133744
DATE           258348
DECIMAL          9821
DIGIT            5442
ELECTRONIC       5162
FRACTION         1196
LETTERS        152790
MEASURE         14783
MONEY            6128
ORDINAL         12703
PLAIN         7353647
PUNCT         1880507
TELEPHONE        4024
TIME             1465
Name: class, dtype: int64

In [8]:
all_data_classes = list(all_data.groupby('class'))

%time
data_balanced_classes = pd.concat([v.sample(min(10000, len(v))) for k, v in all_data_classes])

In [53]:
def data_balance_randomize_classes(max_len=10000):
    global data_balanced_classes
    data_balanced_classes = pd.concat([v.sample(min(max_len, len(v))) for k, v in all_data_classes])

In [79]:
%%time
data_balance_randomize_classes(10000)

CPU times: user 368 ms, sys: 0 ns, total: 368 ms
Wall time: 367 ms


In [55]:
data_balanced_classes.groupby("class")["class"].count()

class
ADDRESS         522
CARDINAL      10000
DATE          10000
DECIMAL        9821
DIGIT          5442
ELECTRONIC     5162
FRACTION       1196
LETTERS       10000
MEASURE       10000
MONEY          6128
ORDINAL       10000
PLAIN         10000
PUNCT         10000
TELEPHONE      4024
TIME           1465
Name: class, dtype: int64

In [11]:
data_balanced_classes.sample(10)

,sentence_id,token_id,class,before,after
4209257,325591,11,CARDINAL,3,three
1500854,118701,15,PUNCT,.,.
379218,30697,8,MONEY,$130M,one hundred thirty million dollars
8377259,638971,3,CARDINAL,5,five
8356962,637474,9,PUNCT,.,.
1090182,86914,4,LETTERS,ABC-,a b c
3848332,298248,3,PUNCT,(,(
7010146,536386,7,ORDINAL,7th,seventh
2984492,232473,1,TELEPHONE,0-679-75096-7,o sil six seven nine sil seven five o nine six...
7899195,603269,0,CARDINAL,5,five


### Word vectors

In [13]:
def load_glove(name):
    with open(name, 'r') as f: lines = [line.split() for line in f]
    words = [d[0] for d in lines]
    vecs = np.stack(np.array(d[1:], dtype=np.float32) for d in lines)
    wordidx = {o:i for i,o in enumerate(words)}
    return vecs, words, wordidx

In [14]:
wv_vecs, wv_words, wv_idx = load_glove('/home/ohu/koodi/data/glove_wordvec/glove.6B.50d.txt')

In [15]:
re_apos = re.compile(r"(\w)'s\b")         # make 's a separate word
re_mw_punc = re.compile(r"(\w[’'])(\w)")  # other ' in a word creates 2 words
re_punc = re.compile("([\"().,;:/_?!—])") # add spaces around punctuation
re_mult_space = re.compile(r"  *")        # replace multiple spaces with just one

def simple_tokeniser(sent):
    sent = re_apos.sub(r"\1 's", sent)
    sent = re_mw_punc.sub(r"\1 \2", sent)
    sent = re_punc.sub(r" \1 ", sent).replace('-', ' - ')
    sent = re_punc.sub(r" \1 ", sent)
    sent = re_mult_space.sub(' ', sent)
    return sent.lower().split()
simple_tokeniser("asdf's   asdf   -testaaa")

['asdf', "'s", 'asdf', '-', 'testaaa']

arr = [simple_tokeniser(s_)[0] for s_ in list(all_data.sample(1000)['before'])]
[s in wv_idx for s in arr].count(True) / 1000

In [74]:
def get_random_sample():
    sample_row = data_balanced_classes.iloc[random.randint(1, len(data_balanced_classes)-1)]
    sentence_id = sample_row['class']

    #rows = all_data[all_data['sentence_id']==sample_row['sentence_id']]
    rows = all_data_sentence_index.loc[sample_row['sentence_id']]
    befores = rows.before.values
        
    token_id_idx = list(rows['token_id']).index(sample_row['token_id'])
    befores[token_id_idx] = '*****'
    str_list = simple_tokeniser(' '.join(befores))
    
    word_vect = np.zeros((1, len(str_list), wv_vecs.shape[1]), dtype=np.float32)
    # var = np.zeros((1, len(str_list), wv_vecs.shape[1]+1))
    for i, w in enumerate(str_list):
        if w=='*****':
            word_vect[0][i] = np.zeros((1, wv_vecs.shape[1]))
        else:
            try:
                word_vect[0][i] = wv_vecs[wv_idx[w]]
            except KeyError:
                word_vect[0][i] = np.random.rand(1, wv_vecs.shape[1])
    return sample_row['before'], sample_row['after'], sample_row['class'], word_vect
            
# get_random_sample()
s_bef, s_aft, s_class, s_word_v = get_random_sample()
print(s_class, ':', s_bef, '->', s_aft, ':', s_word_v.shape, type(s_word_v[0,0,0]))

ADDRESS : B03003 -> b o three o o three : (1, 15, 50) <class 'numpy.float32'>


In [17]:
%%timeit
get_random_sample()

597 µs ± 38.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Categories and Chars

In [18]:
categories_all = all_data["class"].unique()
print(categories_all)
print(len(categories_all))
categories_index = dict((c, i) for i, c in enumerate(categories_all))

['PLAIN' 'PUNCT' 'DATE' 'LETTERS' 'CARDINAL' 'DECIMAL' 'MEASURE' 'MONEY'
 'ORDINAL' 'TIME' 'ELECTRONIC' 'DIGIT' 'FRACTION' 'TELEPHONE' 'ADDRESS']
15


In [19]:
letters_all = sorted(list(set(''.join(all_data['before']))))
letters_index = dict((c, i) for i, c in enumerate(letters_all))
letters_n = len(letters_all)
print(''.join(letters_all))
print(len(letters_all))

 !"#$%&'()+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz|~¡£¥ª«²³µº»¼½¾¿éɒʻˈΩμ—€⅓⅔⅛⅝⅞
112


In [20]:
def string_to_tensor(line):
    tensor = torch.zeros(1, len(line), letters_n)
    for li, letter in enumerate(line):
        tensor[0, li, letters_index[letter]] = 1
    return tensor
print(string_to_tensor('wordup').size())

torch.Size([1, 6, 112])


# Model functions

In [ ]:
use_cuda = False

In [ ]:
class RNN_WORDS_CHARS_CLASS(nn.Module):
    def __init__(self, wordvect_size, letters_size, hidden_size, output_size, n_layers=1):
        super(RNN_WORDS_CHARS_CLASS, self).__init__()

        self.train_iterations = 0
        self.train_history = []
        
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.rnn_words = nn.LSTM(wordvect_size, hidden_size, n_layers, batch_first=True)

        self.rnn_chars = nn.LSTM(letters_size, hidden_size, n_layers, batch_first=True)
        
        #self.lin_1 = nn.Linear(hidden_size*2, 1024)
        self.lin_output = nn.Linear(hidden_size*2, output_size)

    def forward(self, word_vectors, string_tensor, hidden = None, init_hidden = True):
        if init_hidden:
            hidden_words, hidden_chars = self.init_hidden()
        
        all_outputs_words, hidden_words = self.rnn_words(word_vectors, hidden_words)
        output_words = all_outputs_words[:, -1]
        
        all_outputs_chars, hidden_chars = self.rnn_chars(string_tensor, hidden_chars)
        output_chars = all_outputs_chars[:, -1]
        
        output = torch.cat((output_words, output_chars), 1)
        
        #output = self.lin_1(output)
        output = self.lin_output(output)
        output = F.log_softmax(output)
        return output

    def init_hidden(self):
        var1_1 = Variable(torch.zeros(1, self.n_layers, self.hidden_size))
        var1_2 = Variable(torch.zeros(1, self.n_layers, self.hidden_size))
        var2_1 = Variable(torch.zeros(1, self.n_layers, self.hidden_size))
        var2_2 = Variable(torch.zeros(1, self.n_layers, self.hidden_size))
        if use_cuda:
            var1_1 = var1_1.cuda(); var1_2 = var1_2.cuda()
            var2_1 = var2_1.cuda(); var2_2 = var2_2.cuda()
        return ((var1_1, var1_2), (var2_1, var2_2))

use_cuda = False
s_bef, s_aft, s_class, s_word_vs = get_random_sample()
s_string = string_to_tensor(s_bef)
model_tmp = RNN_WORDS_CHARS_CLASS(wordvect_size=s_word_vs.shape[-1], letters_size=letters_n,
                              hidden_size=128, output_size=len(categories_all))
print('Word vect size:', s_word_vs.shape, '. String vector size:', s_string.size())
output = model_tmp(Variable(torch.from_numpy(s_word_vs)), Variable(s_string))
print('Output:', output.size())
use_cuda = True

tmp = use_cuda
use_cuda = True
model_tmp.cuda()
output = model_tmp(Variable(torch.from_numpy(s_word_vs)).cuda(), Variable(s_string).cuda())
use_cuda = tmp
type(output.data)

### Accuracy

In [ ]:
def category_from_output(output):
    top_n, top_i = output.data.topk(1)
    category_i = top_i[0][0]
    return categories_all[category_i], category_i

print(category_from_output(output))

In [ ]:
def test_model_accuracy(model, n_sample=10000):
    model.eval()
    n_correct = 0
    for iteration in range(n_sample):
        s_bef, s_aft, s_class, s_word_vs = get_random_sample()
        s_string = Variable(string_to_tensor(s_bef))
        s_word_vs = Variable(torch.from_numpy(s_word_vs))
        if use_cuda:
            s_word_vs = s_word_vs.cuda()
            s_string = s_string.cuda()
        output = model(s_word_vs, s_string)
        if s_class == category_from_output(output)[0]:
            n_correct += 1

    print("Accuracy: {:>4.2%} ({:>8d}/{:>8d})".format(
            n_correct/n_sample, n_correct, n_sample))

%%time
test_model_accuracy(model_tmp.cuda(), n_sample=1000)

### Training

In [ ]:
def train(model, category, word_vectors, string, loss_function, optimizer):
    category_tensor = Variable(torch.LongTensor([categories_index[category]]))
    word_vectors = Variable(torch.from_numpy(word_vectors))
    string = Variable(string_to_tensor(string))
    if use_cuda:
        category_tensor = category_tensor.cuda()
        word_vectors = word_vectors.cuda()
        string = string.cuda()

    output = model(word_vectors, string)
    loss = loss_function(output, category_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return output, loss.data[0]

loss_function = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
s_bef, s_aft, s_class, s_word_vs = get_random_sample()
train(model, s_class, s_word_vs, s_bef)

In [ ]:
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [ ]:
def train_iterations(n_iters=100000, lr=0.001, print_every=20000, plot_every=1000):
    model.train()
    start = time.time()
    
    loss_function = nn.NLLLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    current_loss = 0
    current_loss_iter = 0

    for iteration in range(1, n_iters + 1):
        model.train_iterations += 1
        s_bef, s_aft, s_class, s_word_vs = get_random_sample()

        output, loss = train(model, s_class, s_word_vs, s_bef, loss_function, optimizer)
        current_loss += loss
        current_loss_iter += 1

        # Print iter number, loss, name and guess
        if iteration % print_every == 0:
            guess, guess_i = category_from_output(output)
            correct = '✓' if guess == s_class else "✗ ({})".format(s_class)
            print("{:>6d} {:>4.0%} ({:>8}) {:>7.3f}   | {:>6.2f}: {} -> {} {}".format(
                model.train_iterations, iteration/n_iters, timeSince(start),
                current_loss/current_loss_iter, loss,
                s_bef, guess, correct ))

        # Add current loss avg to list of losses
        if iteration % plot_every == 0:
            model.train_history.append((current_loss / plot_every, lr))
            current_loss = 0
            current_loss_iter = 0
    
    test_model_accuracy(model, n_sample=10000)

# Model testing

In [ ]:
model = RNN_WORDS_CHARS_CLASS(wordvect_size=s_word_vs.shape[-1], letters_size=letters_n,
                              hidden_size=128, output_size=len(categories_all)).cuda()

In [ ]:
train_iterations(n_iters=10000, print_every=5000)

In [ ]:
train_iterations(n_iters=200000)

In [ ]:
train_iterations(n_iters=200000, lr=0.0001)

### Results eval

In [ ]:
# Keep track of correct guesses in a confusion matrix
confusion = torch.zeros(len(categories_all), len(categories_all))
n_confusion = 100000

# Go through a bunch of examples and record which are correctly guessed
for i in range(n_confusion):
    s_bef, s_aft, s_class, s_word_vs = get_random_sample()
    word_vectors = Variable(torch.from_numpy(s_word_vs))
    string = Variable(string_to_tensor(s_bef))
    if use_cuda:
        word_vectors = word_vectors.cuda()
        string = string.cuda()
    output = model(word_vects, string)
    guess, guess_i = category_from_output(output)
    category_i = categories_index[s_class]
    confusion[category_i][guess_i] += 1

# Normalize by dividing every row by its sum
for i in range(len(categories_all)):
    confusion[i] = confusion[i] / confusion[i].sum()

# Set up plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([''] + list(categories_all), rotation=90)
ax.set_yticklabels([''] + list(categories_all))

# Force label at every tick
ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()

In [ ]:
all_losses = [arr[0] for arr in model.train_history]
plt.figure()
plt.plot(all_losses)